In [105]:
from flask import Flask, render_template, request, jsonify
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as ureq
import pandas as pd
import pymongo

In [106]:
# main
web_url = 'https://www.flipkart.com/search?q=shoes'
uclient = ureq(web_url)
web_page = uclient.read()
uclient.close()

In [107]:
# main
web_html = bs(web_page, 'html.parser')

In [108]:
# main
big_boxes = web_html.findAll('div', {'class': '_13oc-S'})

In [109]:
# main
links = []
for box in big_boxes:
    shoes = box.findAll('div', {'class': '_1xHGtK _373qXS'})
    for shoe in shoes:
        links.append('https://www.flipkart.com' + shoe.a['href'])

In [84]:
# main
name = []
review = []
for link in links:
    shoe_page = requests.get(link)
    shoe_html = bs(shoe_page.text, 'html.parser')
    comments = shoe_html.findAll('div', {'class': '_16PBlm _2RzJ9n'})
    for comment in comments:
        name.append(comment.find('p', {'class': '_2sc7ZR _2V5EHH _1QgsS5'}).text)
        review.append(comment.find('div', {'class': '_6K-7Co'}).text)

In [110]:
conn_url = 'mongodb://localhost:27017/'
db_name = 'scapped_reviews'

# establishing connection
client = pymongo.MongoClient(conn_url)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [116]:
reviews = client[db_name]
shoes_reviews = 'shoes_reviews'
shoes_reviews = reviews[shoes_reviews]

In [111]:
df = pd.DataFrame({'name': name,
                   'review': review})

In [121]:
json_list = []
for i in range(len(name)):
    tmp = {'_id': i,
           'name': name[i],
           'review': review[i]}
    json_list.append(tmp)

In [124]:
# dumping js to our mongo server
shoes_reviews.insert_many(json_list)